In [ ]:
import random

In [ ]:
import pymongo

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import xarray as xr

In [ ]:
client = pymongo.MongoClient("localhost", 22334)
client

In [ ]:
client.database_names()

In [ ]:
db = client.mturk
db

In [ ]:
db.collection_names()

In [ ]:
coll_names = ['objectome64', 'objectome_imglvl', 'ko_obj24_basic_2ways', 'monkobjectome', 'ko_obj24_basic_2ways_mod_ver2']

In [ ]:
client.server_info()

In [ ]:
trial_level = [
    'ImgData',
    'RT',
    'Response',
    'StimDuration',
    'StimShown'
]

In [ ]:
document_level = [
    'AcceptTime',
    'AssignmentID',
    'Browser',
    'Comment',
    'Condition',
    'CreationTime',
    'Description',
    'Duration',
    'HITTypeID',
    'HITid',
    'IPaddress',
    'Keywords',
    'OpSys',
    'Reward',
    'ScreenHeight',
    'ScreenWidth',
    'Timestamp',
    'Title',
    'URL',
    'Version',
    'WindowHeight',
    'WindowWidth',
    'WorkerID',
    'Zoom'
]

In [ ]:
countings_select = {}
for db_name in ['mturk']:
    db = client[db_name]
    db_counting = {}
    for coll_name in coll_names:
        coll = db[coll_name]
        coll_d = {}
        for doc in coll.find():
            trial_count = len(doc['Response'])
            coll_d[doc['AssignmentID']] = trial_count
        db_counting[coll_name] = coll_d
    countings_select[db_name] = db_counting

countings_select

In [ ]:
for db_k in countings_select:
    db_v = countings_select[db_k]
    for coll_k in db_v:
        coll_v = db_v[coll_k]
        coll_trials_count = 0
        for doc_k in coll_v:
            doc_v = coll_v[doc_k]
            coll_trials_count = coll_trials_count + doc_v
        print('    {}: {} trials'.format(coll_k, coll_trials_count))

In [ ]:
mome = db.monkobjectome
mome

In [ ]:
mome.count()

In [ ]:
momerand_index = random.randrange(mome.count())
momerand = mome.find()[momerand_index]
momerand

In [ ]:
momerand_index

In [ ]:
len(momerand["ImgData"])

In [ ]:
momerand.keys()

In [ ]:
type(momerand)

In [ ]:
[(k, type(momerand[k])) for k in momerand]

In [ ]:
momerand["Qualification"]

In [ ]:
momerand["RT"][:5]

In [ ]:
momerand["Response"][:5]

In [ ]:
momerand["StimShown"][:5]

In [ ]:
momerand["ImgData"][:5]

In [ ]:
[len(momerand[k]) for k in trial_level]

In [ ]:
sample_index = 123
[momerand[k][sample_index] for k in trial_level]

In [ ]:
def strip_response(response):
    return response.replace("_label", "")

strip_response(momerand["Response"][sample_index])

In [ ]:
momerand["ImgData"][sample_index][0]["obj"]

In [ ]:
momerand["ImgData"][sample_index][0]["obj"] == strip_response(momerand["Response"][sample_index])

In [ ]:
def correct_response(doc, trial_index):
    target = doc["ImgData"][trial_index][0]["obj"]
    response = strip_response(doc["Response"][trial_index])
    return target == response

correct_response(momerand, sample_index)

In [ ]:
names_simple_trial_level = ["RT", "Response", "StimDuration"]

In [ ]:
names_stim_shown = ["Stimulus", "Choice_1", "Choice_2"]

In [ ]:
def df_from_doc(doc):
    df_doc = pd.DataFrame([x[0] for x in doc["ImgData"]])
    for name in names_simple_trial_level:
        df_doc[name] = doc[name]
    for i, name in enumerate(names_stim_shown):
        df_doc[name] = [x[i] for x in doc["StimShown"]]
    for name in document_level:
        df_doc[name] = [doc[name]] * len(df_doc)
    return df_doc

In [ ]:
df_momerand = df_from_doc(momerand)
df_momerand

In [ ]:
def s_from_doc(doc):
    l_correct = [correct_response(doc, i) for i in range(len(doc["ImgData"]))]
    df_doc = df_from_doc(doc)
    levels = [df_doc[name] for name in df_doc.columns]
    mi_doc = pd.MultiIndex.from_arrays(arrays=levels, names=df_doc.columns)
    s_doc = pd.Series(data=l_correct, index=mi_doc)
    return s_doc

In [ ]:
s_momerand = s_from_doc(momerand)
s_momerand

In [ ]:
s_mome = pd.concat([s_from_doc(doc) for doc in mome.find()])
s_mome

In [ ]:
len(s_mome)

In [ ]:
s_mome.index

In [ ]:
type(s_mome.index)

In [ ]:
da_mome = xr.DataArray(s_mome)
da_mome

In [ ]:
da_mome.to_netcdf("monkobjectome_behavior.nc")